# ghen parser to mongodb

In [1]:
import re
import pymongo
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient
from dateutil.relativedelta import relativedelta
%matplotlib inline

### Url set, then use bs4 to get page

In [2]:
# TODAY DATE
now = datetime.datetime.now()
# TODAY_ymd
today_ymd = datetime.date.today()     # datetime format
# today_ymd = now.strftime("%Y-%m-%d") # str format


# PARAMETER 
domain = 'https://www.google.com.tw/'
title_list = []
link_list = []
source_list = []
date_list = []

url = 'https://www.google.com.tw/search?q=%E6%AF%94%E7%89%B9%E5%B9%A3&as_occt=title&tbm=nws&num=100&as_sitesearch=http://news.cnyes.com'

### Google Scrape_Parser Functions

In [4]:
# > parse main google-search-main-page
def google_search_parse(url):
    
    print('->正在抓取頁面', url)
    next_page = ''
    
    # request and response 
    res = requests.get(url)
    content = res.content
    soup = BeautifulSoup(content, "html.parser")
    
    # parse data into list 
    for item in soup.select('.g'):
        
        # data parse position
        title = item.a.text
        link = item.a.attrs['href'].split('q=')[1].split('&sa')[0]
        source = item.span.text.split('-')[0]
        date = google_search_date_transfer(item.span.text.split('-')[1])
        #print(title, date)
        
        # data into list 
        title_list.append(title)
        link_list.append(link)
        source_list.append(source)
        date_list.append(date)
        
    # next_page 
    if soup.select('a.fl'):
        for nav in soup.select('a.fl'):
            if nav.text == '下一頁':
                next_page = domain + nav.attrs['href']
                print('現在切換頁面為', next_page)
        if next_page:
            try:
                parse_google_search(next_page)
            except:
                print('next_page:', next_page)
                print('END')
    else:
        print('爬蟲end.')
            
# > Google Search Time Transfer
def google_search_date_transfer(google_date):
    # 有bug...沒想到換日問題Orz...=_=
    if ('小時' or '分鐘' or '秒') in google_date:
        return str(today_ymd)
    elif '天前' in google_date:
        d = int(google_date.split(' 天前')[0])
        p_date = str(today_ymd - relativedelta(days = d))
        return p_date
    elif '年' in google_date:
        y = int(google_date.split('年')[0])
        m = int(google_date.split('年')[1].split('月')[0])
        d = int(google_date.split('年')[1].split('月')[1].split('日')[0])
        p_date = datetime.datetime(y, m, d).strftime("%Y-%m-%d")
        return p_date
    else:
        p_date = 'no data'
        return p_date

### Catch data

In [5]:
# 抓取google search頁面文章之title, link, date, source
google_search_parse(url)

->正在抓取頁面 https://www.google.com.tw/search?q=%E6%AF%94%E7%89%B9%E5%B9%A3&as_occt=title&tbm=nws&num=100&as_sitesearch=http://news.cnyes.com
現在切換頁面為 https://www.google.com.tw//search?q=allintitle:+%E6%AF%94%E7%89%B9%E5%B9%A3+site:http://news.cnyes.com&num=100&ie=UTF-8&tbm=nws&ei=F8vBWd3sGsvO0ASXw4mgCA&start=100&sa=N
next_page: https://www.google.com.tw//search?q=allintitle:+%E6%AF%94%E7%89%B9%E5%B9%A3+site:http://news.cnyes.com&num=100&ie=UTF-8&tbm=nws&ei=F8vBWd3sGsvO0ASXw4mgCA&start=100&sa=N
END


### to pd.dataframe

In [6]:
# show data
all_dict = {'Title':title_list,
        'Source':source_list,
        'Link':link_list,
        'Date':date_list
       }

In [7]:
df = pd.DataFrame(all_dict) 
df

,Date,Link,Source,Title
0,2017-09-20,http://news.cnyes.com/news/id/3922041,鉅亨網,交易人逃離中國日本取代美國成為最大比特幣交易市場
1,2017-09-20,http://news.cnyes.com/news/id/3921615,鉅亨網,比特幣重回4000美元對中國監管影響憂慮降低
2,2017-09-18,http://news.cnyes.com/news/id/3920558,鉅亨網,還沒完！中國傳斷比特幣海外交易阻洗黑錢
3,2017-09-15,http://news.cnyes.com/news/id/3919088,鉅亨網,傳比特幣交易平台在中國將全部關停並於近期退出市場
4,2017-09-14,http://news.cnyes.com/news/id/3918402,鉅亨網,比特幣直直落大陸炒家堅不清倉新玩法：做空它
5,2017-09-20,http://news.cnyes.com/news/id/3922338,鉅亨網,世界最大避險基金創辦人說比特幣就是一個泡沫
6,2017-09-14,http://news.cnyes.com/news/id/3918345,鉅亨網,戴蒙稱騙局一場比特幣價格下跌8%
7,2017-09-16,http://news.cnyes.com/news/id/3919535,鉅亨網,中國下重手比特幣暴跌創意股價一度跌停
8,2017-09-05,http://news.cnyes.com/news/id/3910416,鉅亨網,中國大陸禁止以加密貨幣籌資比特幣再下跌200美元
9,2017-08-28,http://news.cnyes.com/news/id/3903981,鉅亨網,〈比特幣〉比特幣是貨幣嗎？墨西哥央行行長：其實是商品


### 抓各自文章內容

In [8]:
Content_list = []

In [9]:
for index, row in df.iterrows():
    x = index
    url = row['Link']
    res = requests.get(url)
    content = res.content
    soup = BeautifulSoup(content, "html.parser")
    for d in soup.findAll("div",{"itemprop":"articleBody"}):
        try:
            Content_list.append(d.text)
        except:
            print('error', index)

In [10]:
df['Content'] = Content_list

In [11]:
df.head()

,Date,Link,Source,Title,Content
0,2017-09-20,http://news.cnyes.com/news/id/3922041,鉅亨網,交易人逃離中國日本取代美國成為最大比特幣交易市場,日本在本周末的比特幣交易量，已經達到全球的 51%，正式取代美國成為最大的市場。\n隨著中國...
1,2017-09-20,http://news.cnyes.com/news/id/3921615,鉅亨網,比特幣重回4000美元對中國監管影響憂慮降低,比特幣今年的價格走勢，依舊像雲霄飛車般快速下沉後又急速爬升。\n據《彭博》報導，比特幣價格週...
2,2017-09-18,http://news.cnyes.com/news/id/3920558,鉅亨網,還沒完！中國傳斷比特幣海外交易阻洗黑錢,中國虛擬貨幣遭遇到最嚴監管，3 大比特幣平台的《火幣網》、《OKCoin 幣行》及《比特幣中...
3,2017-09-15,http://news.cnyes.com/news/id/3919088,鉅亨網,傳比特幣交易平台在中國將全部關停並於近期退出市場,據中國財經媒體《第一財經》報導，週四 (14 日) 傍晚最新消息指出，熟悉中國互聯網金融監管...
4,2017-09-14,http://news.cnyes.com/news/id/3918402,鉅亨網,比特幣直直落大陸炒家堅不清倉新玩法：做空它,摩根大通 CEO 戴蒙 (Jamie Dimon) 抨擊比特幣 (Bitcoin) 為騙局後...


### 整理時間格式

In [12]:
for index, row in df.iterrows():
    try:
        x = datetime.datetime.strptime(row['Date'], '%Y-%m-%d')
        df.set_value(index, 'Date', x)
    except:
        print(index, 'error')

### list，用於存入db

In [13]:
tt = []
for index, row in df.iterrows():
    temp = {
        "Date": row['Date'],
        "Link": row['Link'],
        "Source": row['Source'],
        "Title": row['Title'],
        "Content": row['Content']
    }
    tt.append(temp)

### to mongodb

In [14]:
client = MongoClient('mongodb://localhost:27017/')
db = client.bitcoinnews_db
for item in tt:
    db.newsCollection.insert_one(item)